<a href="https://colab.research.google.com/github/Ritwik-1/Fine-Tune-SAM/blob/master/SAM_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything.git

In [3]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers,regularizers
from tensorflow.keras.datasets import mnist

In [4]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

model_checkpoint = "/content/drive/MyDrive/Colab Notebooks/assets/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint = model_checkpoint)


In [9]:
print(type(sam))

sam.image_encoder.training = False

for name, param in sam.named_parameters():
    print(name, param.requires_grad)


<class 'segment_anything.modeling.sam.Sam'>
image_encoder.pos_embed True
image_encoder.patch_embed.proj.weight True
image_encoder.patch_embed.proj.bias True
image_encoder.blocks.0.norm1.weight True
image_encoder.blocks.0.norm1.bias True
image_encoder.blocks.0.attn.rel_pos_h True
image_encoder.blocks.0.attn.rel_pos_w True
image_encoder.blocks.0.attn.qkv.weight True
image_encoder.blocks.0.attn.qkv.bias True
image_encoder.blocks.0.attn.proj.weight True
image_encoder.blocks.0.attn.proj.bias True
image_encoder.blocks.0.norm2.weight True
image_encoder.blocks.0.norm2.bias True
image_encoder.blocks.0.mlp.lin1.weight True
image_encoder.blocks.0.mlp.lin1.bias True
image_encoder.blocks.0.mlp.lin2.weight True
image_encoder.blocks.0.mlp.lin2.bias True
image_encoder.blocks.1.norm1.weight True
image_encoder.blocks.1.norm1.bias True
image_encoder.blocks.1.attn.rel_pos_h True
image_encoder.blocks.1.attn.rel_pos_w True
image_encoder.blocks.1.attn.qkv.weight True
image_encoder.blocks.1.attn.qkv.bias True